In [48]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nextworld/1661-0.txt


# Import necessary Libraries

In [49]:
import tensorflow as tf
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import time
import gc
import sys

print(f"Tensorflow Version: {tf.__version__}")
print(f"Pandas Version: {pd.__version__}")
print(f"Numpy Version: {np.__version__}")
print(f"System Version: {sys.version}")

mpl.rcParams['figure.figsize'] = (17, 5)
mpl.rcParams['axes.grid'] = False
sns.set_style("whitegrid")

notebookstart= time.time()

Tensorflow Version: 2.13.0
Pandas Version: 2.0.3
Numpy Version: 1.24.3
System Version: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]


# Read the Data

In [50]:
path = '/kaggle/input/nextworld/1661-0.txt'
data = open(path).read().lower()
print('corpus length:', len(data))

corpus length: 581888


In [51]:
print(data[:500])

﻿
project gutenberg's the adventures of sherlock holmes, by arthur conan doyle

this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  you may copy it, give it away or
re-use it under the terms of the project gutenberg license included
with this ebook or online at www.gutenberg.net


title: the adventures of sherlock holmes

author: arthur conan doyle

release date: november 29, 2002 [ebook #1661]
last updated: may 20, 2019

language: english

charac


Create a Tokenizer class

In [52]:
from keras.preprocessing.text import Tokenizer

In [53]:
token = Tokenizer()
token.fit_on_texts([data])
encoded = token.texts_to_sequences([data])[0]
encoded

[4789,
 145,
 4790,
 1,
 1020,
 4,
 128,
 34,
 45,
 611,
 2235,
 2236,
 30,
 1021,
 15,
 23,
 1,
 275,
 4,
 394,
 2237,
 21,
 51,
 1676,
 2,
 18,
 572,
 51,
 3398,
 3399,
 13,
 75,
 817,
 10,
 213,
 10,
 124,
 63,
 2238,
 275,
 10,
 262,
 1,
 480,
 4,
 1,
 145,
 130,
 655,
 2239,
 18,
 30,
 1021,
 63,
 2240,
 21,
 1093,
 130,
 3400,
 2666,
 1,
 1020,
 4,
 128,
 34,
 4791,
 611,
 2235,
 2236,
 4792,
 1022,
 4793,
 4794,
 4795,
 1021,
 2241,
 139,
 3401,
 75,
 3402,
 4796,
 3403,
 1094,
 573,
 243,
 4797,
 4798,
 1469,
 722,
 4,
 30,
 145,
 130,
 1021,
 1,
 1020,
 4,
 128,
 34,
 1928,
 45,
 50,
 3404,
 145,
 130,
 2667,
 2,
 3405,
 3406,
 1677,
 1,
 1020,
 4,
 128,
 34,
 45,
 611,
 2235,
 2236,
 1929,
 6,
 5,
 885,
 8,
 940,
 2668,
 1,
 248,
 481,
 691,
 2669,
 5,
 113,
 4,
 2242,
 3407,
 1,
 774,
 1470,
 522,
 3408,
 1,
 312,
 1023,
 941,
 3409,
 1,
 58,
 18,
 1,
 1095,
 942,
 3410,
 1,
 612,
 4,
 1,
 448,
 1471,
 3411,
 1,
 612,
 4,
 1,
 1930,
 886,
 3412,
 1,
 612,
 4,
 1,
 3413,
 692

Calculate the vocabulary size by adding 1 to the length of the word index in the 'token' object

In [54]:
vocab = len(token.word_index) +1
print("Vocab size: " ,vocab)

Vocab size:  8932


 Create an empty list to store sequences

In [55]:
sequences = list()
for i in range(1, len(encoded)):
    sequence = encoded[i-1:i+1]
    sequences.append(sequence)
print("total: ", len(sequences))

total:  111252


In [56]:
sequences 

[[4789, 145],
 [145, 4790],
 [4790, 1],
 [1, 1020],
 [1020, 4],
 [4, 128],
 [128, 34],
 [34, 45],
 [45, 611],
 [611, 2235],
 [2235, 2236],
 [2236, 30],
 [30, 1021],
 [1021, 15],
 [15, 23],
 [23, 1],
 [1, 275],
 [275, 4],
 [4, 394],
 [394, 2237],
 [2237, 21],
 [21, 51],
 [51, 1676],
 [1676, 2],
 [2, 18],
 [18, 572],
 [572, 51],
 [51, 3398],
 [3398, 3399],
 [3399, 13],
 [13, 75],
 [75, 817],
 [817, 10],
 [10, 213],
 [213, 10],
 [10, 124],
 [124, 63],
 [63, 2238],
 [2238, 275],
 [275, 10],
 [10, 262],
 [262, 1],
 [1, 480],
 [480, 4],
 [4, 1],
 [1, 145],
 [145, 130],
 [130, 655],
 [655, 2239],
 [2239, 18],
 [18, 30],
 [30, 1021],
 [1021, 63],
 [63, 2240],
 [2240, 21],
 [21, 1093],
 [1093, 130],
 [130, 3400],
 [3400, 2666],
 [2666, 1],
 [1, 1020],
 [1020, 4],
 [4, 128],
 [128, 34],
 [34, 4791],
 [4791, 611],
 [611, 2235],
 [2235, 2236],
 [2236, 4792],
 [4792, 1022],
 [1022, 4793],
 [4793, 4794],
 [4794, 4795],
 [4795, 1021],
 [1021, 2241],
 [2241, 139],
 [139, 3401],
 [3401, 75],
 [75, 3402

# Sequences Extraction

In [57]:
from numpy import array

In [58]:
sequences = array(sequences)
x,y = sequences[:,0], sequences[:,1]

In [59]:
x[:5]

array([4789,  145, 4790,    1, 1020])

In [60]:
y[:5]

array([ 145, 4790,    1, 1020,    4])

In [61]:
from keras.utils import to_categorical

In [62]:
y = to_categorical(y, num_classes = vocab)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

# Model

In [63]:
pip install keras tensorflow

Note: you may need to restart the kernel to use updated packages.


In [64]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense


In [65]:
model = Sequential()
model.add(Embedding(vocab, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab, activation="softmax"))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 1, 10)             89320     
                                                                 
 lstm_1 (LSTM)               (None, 50)                12200     
                                                                 
 dense_1 (Dense)             (None, 8932)              455532    
                                                                 
Total params: 557052 (2.12 MB)
Trainable params: 557052 (2.12 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


# Model Compile

In [66]:
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])

In [67]:
model.fit(x,y,epochs=100)

Epoch 1/100
3477/3477 [==============================] - 34s 9ms/step - loss: 6.6992 - accuracy: 0.0512
Epoch 2/100
3477/3477 [==============================] - 32s 9ms/step - loss: 6.2481 - accuracy: 0.0551
Epoch 3/100
3477/3477 [==============================] - 32s 9ms/step - loss: 5.9849 - accuracy: 0.0773
Epoch 4/100
3477/3477 [==============================] - 32s 9ms/step - loss: 5.6974 - accuracy: 0.1155
Epoch 5/100
3477/3477 [==============================] - 32s 9ms/step - loss: 5.5019 - accuracy: 0.1316
Epoch 6/100
3477/3477 [==============================] - 32s 9ms/step - loss: 5.3584 - accuracy: 0.1399
Epoch 7/100
3477/3477 [==============================] - 32s 9ms/step - loss: 5.2511 - accuracy: 0.1459
Epoch 8/100
3477/3477 [==============================] - 32s 9ms/step - loss: 5.1660 - accuracy: 0.1518
Epoch 9/100
3477/3477 [==============================] - 32s 9ms/step - loss: 5.0952 - accuracy: 0.1567
Epoch 10/100
3477/3477 [==============================] - 32s 9m

In [68]:
model.compile(loss='your_loss_function', optimizer='adam', metrics=['accuracy'])

In [70]:
import numpy as np
from numpy import array
from keras.optimizers import Adam

# Assuming you have defined the 'model' and 'token' objects somewhere before calling the function

def abc(model, token, inputt, npred):
    in_text, result = inputt, inputt
    for _ in range(npred):
        encoded = token.texts_to_sequences([in_text])[0]
        encoded = array(encoded)

        # Reshape the input to match the model's input shape
        encoded = np.reshape(encoded, (1, len(encoded)))

        # Compile the model before making predictions
        optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate if needed
        model.compile(loss='your_loss_function', optimizer=optimizer, metrics=['accuracy'])

        # Ensure that the model's architecture is compatible with the input shape
        # If you have variable input shapes, you may need to adjust this part accordingly
        if len(encoded[0]) > 0:
            yhat_probs = model.predict(encoded)
            yhat = np.argmax(yhat_probs)

            out_word = ""
            for word, index in token.word_index.items():
                if index == yhat:
                    out_word = word
                    break
            in_text = out_word   # Update the input with the predicted word
            result += ' ' + out_word
        else:
            print("Warning: Empty input sequence. Please check the input data.")

    return result

# Replace 'your_loss_function' with the appropriate loss function for your problem
# Make sure 'model' and 'token' are defined before calling the function
print(abc(model, token, 'this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  you', 3))


1/1 [==============================] - 0s 394ms/step
this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.  you not be a
